In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo

client = pymongo.MongoClient('mongodb://localhost:27017')
db = client.charities_db
collection = db.charities

def init_scaper():
    # URL of page to be scraped
    url = 'https://www.raptim.org/11-international-ngos-fighting-malaria/'
    # Retrieve page with the requests module
    response = requests.get(url)
    # Create BeautifulSoup object; parse with 'lxml'
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

def scrape_charity_data(soup):
    charity_items = []
    results = soup.find('article', class_='right')

    headers = results.find_all('h3')
    for header in headers:
        header_text = header.find('strong').text

        p = header.next_sibling
        a = p.find('a')
        if a:
            link = a['href']
        else:
            link = None

        charity_items.append({
            'text': header_text,
            'link': link
        })

    return charity_items

if __name__ == '__main__':
    soup = init_scaper()
    charity_items = scrape_charity_data(soup)
    for item in charity_items:
        print(item)

collection.insert_many(charity_items)

charities =db.charities.find()
for charity in charities:
    print(charity)

